In [ ]:
# python

import csv
import json
from pathlib import Path
import requests

BASE_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"



In [ ]:
query = "((COVID-19) AND 2026 [pdat])"

def get_search_context(query: str) -> tuple[str, str]:

    params = {

    "db": "pmc",

    "term": query,

    "retmode": "json",

    "usehistory": "y",

    "retmax": 0,

    }

    response = requests.get(f"{BASE_URL}/esearch.fcgi", params=params, timeout=30)

    response.raise_for_status()

    data: dict = response.json().get("esearchresult", {})

    return data.get("webenv", None), data.get("querykey", None), data.get("count", None)

In [ ]:
get_search_context(query=query)

In [ ]:
params = {

    "db": "pmc",

    "term": query,

    "retmode": "json",

    "usehistory": "y",

    "retmax": 0,

    }

response = requests.get(f"{BASE_URL}/esearch.fcgi", params=params, timeout=30)

response.raise_for_status()

data: dict = response.json().get("esearchresult", {})

In [ ]:
data

In [ ]:
params = {
    "db": "pmc",   # ← change here
    "term": query,
    "retmode": "json",
    "usehistory": "y",
    "retmax": 100,
}

response = requests.get(f"{BASE_URL}/esearch.fcgi", params=params, timeout=30)
response.raise_for_status()

data = response.json().get("esearchresult", {})

pmids = data.get("idlist", [])

print(pmids)

In [ ]:
pmid = "12913732"

params = {
    "db": "pmc",
    "id": pmid,
    "retmode": "json"
}

response = requests.get(BASE_URL + "esummary.fcgi", params=params)
response.raise_for_status()

data = response.json()

In [ ]:
data

In [ ]:
def parse_pubmed_esummary(results: dict, pmid: str) -> dict:
    """
    Parse the results for one specific pmid
    From the pubmed esummary return
    Return the dictionary of needed data
    """

    article = results.get("result", {}).get(pmid)
    if not article:
        return {}

    # Authors
    authors = article.get("authors", [])
    author_names = [a.get("name", "") for a in authors if "name" in a]

    # Article IDs
    doi = ""
    pmcid = ""
    pubmed_id = ""

    for aid in article.get("articleids", []):
        if aid.get("idtype") == "doi":
            doi = aid.get("value", "")
        if aid.get("idtype") == "pmcid":
            pmcid = aid.get("value", "")
        if aid.get("idtype") == "pmid":
            pubmed_id = aid.get("value", "")

    # Dates
    pubdate = article.get("pubdate", "")
    epubdate = article.get("epubdate", "")
    printpubdate = article.get("printpubdate", "")
    sortdate = article.get("sortdate", "")
    pmc_live_date = article.get("pmclivedate", "")

    resolved_date = epubdate or pubdate or printpubdate or sortdate

    return {
        "uid": pmid,                 # the key used in esummary
        "pubmed_id": pubmed_id,      # the true PMID from articleids
        "title": article.get("title", ""),
        "journal": article.get("fulljournalname", ""),
        "journal_abbrev": article.get("source", ""),
        "pubdate": pubdate,
        "epubdate": epubdate,
        "printpubdate": printpubdate,
        "sortdate": sortdate,
        "pmc_live_date": pmc_live_date,
        "resolved_pubdate": resolved_date,
        "volume": article.get("volume", ""),
        "issue": article.get("issue", ""),
        "pages": article.get("pages", ""),
        "doi": doi,
        "pmcid": pmcid,
        "authors": "; ".join(author_names),
        "author_count": len(author_names),
        "publication_types": "; ".join(article.get("pubtype", [])),
        "language": "; ".join(article.get("lang", [])),
    }

In [ ]:
parse_pubmed_esummary(data, pmid)